# Convolutional Neural Network

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.examples.tutorials.mnist import input_data

## data set

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)
X_trn, Y_trn, X_test, Y_test = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
X_trn = X_trn.reshape(-1, 28, 28, 1)  # 28x28x1 input img
X_test = X_test.reshape(-1, 28, 28, 1)  # 28x28x1 input img

## make placeholder for inputs and labels

In [ ]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.int32, [None])

## Help functions

In [ ]:
def init_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.01))

## model graphs
### tf.nn.conv2d(input, filter, strides, padding)
- input tensor shape: [batch, in_height, in_width, in_channels]
- filter tensor shape: [filter_height, filter_width, in_channels, out_channels]
- strid tensor: input의 각 dimension에 대한 sliding window의 stride
- padding: "SAME" or "VALID"

### tf.nn.max_pool(value, k_size, strides, padding)
- value: A 4-D Tensor with shape [batch, height, width, channels] and type tf.float32
- k_size: A list of integers that has length >=4. The size of the window for each dimension of the input tensor
- strides: A list of integers that has lenght >=4. The stride of the sliding window for dimension of the input tensor.
- padding: "SAME", "VALID"

### tf.nn.relu(features)
- max(features, 0)

### tf.nn.dropout(x, keep_prob)
- keep_prob: dropout probability

In [ ]:
def model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden):
    l1a = tf.nn.relu(tf.nn.conv2d(X, w,
                                  strides=[1, 1, 1, 1], padding='SAME'))
    # l1a output shape=(?, input_height, input_width, number_of_channels_layer1)
    l1 = tf.nn.max_pool(l1a, ksize=[1, 2, 2, 1],              
                        strides=[1, 2, 2, 1], padding='SAME')
    # l1 output shape=(?, input_height/2, input_width/2, number_of_channels_layer1)
    l1 = tf.nn.dropout(l1, p_keep_conv)
                                  
    l2a = tf.nn.relu(tf.nn.conv2d(l1, w2,                     
                        strides=[1, 1, 1, 1], padding='SAME'))
    # l2a output shape=(?, input_height/2, input_width/2, number_of_channels_layer2)
    l2 = tf.nn.max_pool(l2a, ksize=[1, 2, 2, 1],              
                        strides=[1, 2, 2, 1], padding='SAME')
    # l2 shape=(?, input_height/4, input_width/4, number_of_channels_layer2)
    l2 = tf.nn.dropout(l2, p_keep_conv)

    l3a = tf.nn.relu(tf.nn.conv2d(l2, w3,                     
                        strides=[1, 1, 1, 1], padding='SAME'))
    # l3a shape=(?, input_height/4, input_width/4, number_of_channels_layer3)
    l3 = tf.nn.max_pool(l3a, ksize=[1, 2, 2, 1],              
                        strides=[1, 2, 2, 1], padding='SAME')
    # l3 shape=(?, input_height/8, input_width/8, number_of_channels_layer3)
    l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])
    # flatten to (?, input_height/8 * input_width/8 * number_of_channels_layer3)
    l3 = tf.nn.dropout(l3, p_keep_conv)

    l4 = tf.nn.relu(tf.matmul(l3, w4))
    #fully connected_layer
    l4 = tf.nn.dropout(l4, p_keep_hidden)
    
    pyx = tf.matmul(l4, w_o)
    return pyx

## weight variable & Model
### convlution + max pooling layer
* 1st convolution + max pooling layer: 3 X 3  Convolution, 2 * 2 max pooling, channel_in: 1, channel_out: 32
* 2nd convolution + max pooling layer: 3 X 3  Convolution, 2 * 2 max pooling, channel_in: 32, channel_out: 64
* 3th convolution + max pooling layer: 3 X 3  Convolution, 2 * 2 max pooling, channel_in: 64, channel_out: 128

### fully connected layer
* flatten input layer to the fully connected layer
* hidden layer unit: 625
* output layer: number of classes (10)

### dropout
* convolution layer dropout
* fully connected layer dropout

### Weight 

In [ ]:
w = init_weights([3, 3, 1, 32])
w2 = init_weights([3, 3, 32, 64])
w3 = init_weights([3, 3, 64, 128])
w4 = init_weights([128 * 4 * 4, 625])
w_o = init_weights([625, 10])

In [ ]:
p_keep_conv = tf.placeholder(tf.float32)
p_keep_hidden = tf.placeholder(tf.float32)
py_x = model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)

### loss function

In [ ]:
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=py_x, labels=Y))

### optimizer

In [ ]:
train_op = tf.train.AdagradOptimizer(learning_rate=0.1).minimize(loss)

### Accuracy

In [ ]:
correct = tf.nn.in_top_k(py_x, Y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

## Run

In [ ]:
trn_loss_list = []
test_loss_list = []
test_accuracy_list = []
batch_size = 128
with tf.Session() as sess:    
    tf.global_variables_initializer().run()

    for i in range(50):
        trn_loss = 0
        for start, end in zip(range(0, len(X_trn), batch_size), range(batch_size, len(X_trn)+1, batch_size)):
            tmp_loss, _ = sess.run([loss, train_op], 
                                   feed_dict={X: X_trn[start:end], 
                                              Y: Y_trn[start:end], 
                                              p_keep_conv: 1.0, 
                                              p_keep_hidden: 0.8})
            trn_loss += tmp_loss
            
        trn_loss_list.append(trn_loss / batch_size)
        test_loss, test_acc = sess.run([loss, accuracy], 
                                       feed_dict={X: X_test, 
                                                  Y: Y_test, 
                                                  p_keep_conv: 1.0, 
                                                  p_keep_hidden: 1.0})
        test_loss_list.append(test_loss)
        test_accuracy_list.append(test_acc)
        print("epoch: {} test accuracy {:0.4f}".format(i, test_acc))

## Loss plot

In [ ]:
x = np.arange(50)
plt.plot(x, trn_loss_list)
plt.plot(x, test_loss_list)
plt.title("cross entropy loss")
plt.legend(["train loss", "test_loss"])
plt.xlabel("epoch")
plt.ylabel("cross entropy")

In [ ]:
x = np.arange(50)
plt.plot(x, test_accuracy_list)
plt.title("prediction accuracy")
plt.xlabel("epoch")
plt.ylabel("accuracy")